In [8]:

import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np

## Real time testing

In [9]:
### load model
model_path = r"model\ep_range(0, 50)_lr0001.h5"
model= load_model(model_path)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23625344  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 100)               13107300  
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 3)                 303       
                                                                 
Total params: 36,743,047
Trainable params: 36,689,927
Non-trainable params: 53,120
_________________________________________________________________


In [10]:
img_interval = 5
IMG_WIDTH = 256
IMG_HEIGHT = 256
CHANNEL = 3

In [11]:
def prepare_frame(image):
    
    img = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))
    
    #convert to tf and normalize
    tf_img = tf.cast(img, tf.float32)
    tf_img = (tf_img/127.5)-1


    return tf_img

## Sampel prediction
def image_stack(image_list):
    width, height,_ = image_list[0].shape
    stacked_image = np.zeros((IMG_WIDTH ,IMG_HEIGHT,len(image_list)*CHANNEL), np.uint8)
    
    start_index = 0
    for i,cur_img in enumerate(image_list):
        end_index = start_index + CHANNEL

        stacked_image[:,:,start_index:end_index]=cur_img
        start_index = end_index
    return stacked_image

In [12]:
import tensorflow as tf
import numpy as np
import cv2 

video_path = r"E:\data_share_ths\dataset\UCF-101\UCF-101\ApplyEyeMakeup\v_ApplyEyeMakeup_g02_c04.avi"
video_path = r"E:\data_share_ths\dataset\UCF-101\UCF-101\ApplyLipstick\v_ApplyLipstick_g14_c01.avi"
video_path = r"E:\data_share_ths\dataset\UCF-101\UCF-101\Archery\v_Archery_g08_c02.avi"

In [13]:
classMap = ["ApplyEyeMakeup","ApplyLipstick","Archery"]

In [14]:
cap = cv2.VideoCapture(video_path)
count = 0
buffer_frame= []
predicted_action = None

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    


    count = count+1

    tf_image = prepare_frame(frame)
    buffer_frame.append(tf_image)
    print(len(buffer_frame))
    if len(buffer_frame)%img_interval ==0:
        
        stack_image = image_stack(buffer_frame)
        buffer_frame = []
        stack_image = np.array([stack_image]) 
        predicted_result = model.predict(stack_image,batch_size=1)

        index = (np.argmax(predicted_result))

        predicted_action = (classMap[index])
    else:
        buffer_frame.append(tf_image)

    #print(predicted_action)
    cv2.putText(frame,predicted_action, (0,200), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    cv2.imshow('Real time video based action recognition ', frame)

    if cv2.waitKey(1) == ord('q'):
        break


print("Total Frame :",count)
cap.release()
cv2.destroyAllWindows()

1
3
5
1/1 [==============================] - 3s 3s/step
1
3
5
1/1 [==============================] - 0s 23ms/step
1
3
5
1/1 [==============================] - 0s 22ms/step
1
3
5
1/1 [==============================] - 0s 22ms/step
1
3
5
1/1 [==============================] - 0s 24ms/step
1
3
5
1/1 [==============================] - 0s 22ms/step
1
3
5
1/1 [==============================] - 0s 24ms/step
1
3
5
1/1 [==============================] - 0s 24ms/step
1
3
5
1/1 [==============================] - 0s 22ms/step
1
3
5
1/1 [==============================] - 0s 26ms/step
1
3
5
1/1 [==============================] - 0s 22ms/step
1
3
5
1/1 [==============================] - 0s 21ms/step
1
3
5
1/1 [==============================] - 0s 23ms/step
1
3
5
1/1 [==============================] - 0s 21ms/step
1
3
5
1/1 [==============================] - 0s 20ms/step
1
3
5
1/1 [==============================] - 0s 22ms/step
1
3
5
1/1 [==============================] - 0s 22ms/step
1
3
5
1/1 [=====